In [ ]:
import os
import glob

sacred_dir = "logs"
parent_dir = os.getcwd()
print(os.getcwd())
test_json_files = glob.glob(os.path.join(parent_dir, sacred_dir, "**/test**/*info.json"), recursive=True)
print(len(test_json_files))

In [ ]:
import json

def exp_name_from_file(file):
    # test_10000_qvmix_uc_50_owf_25_9__2023-03-15_23-09-01__2023-03-21_06-07-53
    # test_10000_qvmix_uc_50_owf_25_cc_True_5__2023-04-01_23-05-55__2023-04-10_17-30-24
    exp_name = os.path.basename(os.path.dirname(file))
    exp_name = exp_name.split("_")[2:]

    if 'owf' in exp_name and 'dqn' in exp_name:
        index_env = 5
    elif "owf" in exp_name:
        index_env = 4
    else:
        index_env = 5

    if exp_name[index_env] != exp_name[2] and index_env is not 4:
        if int(exp_name[2]) == 10 and int(exp_name[index_env]) < 10:
            pass
        else:
            if not "dqn" in exp_name:
                print("merde")
                print(int(exp_name[2]) == 10, int(exp_name[index_env]) < 10)
                print(exp_name, exp_name[2], exp_name[index_env])
    if "cc_True" in file:
        exp_name = [exp_name[0], exp_name[index_env-1], exp_name[index_env], "cc_True"]
    else:
        exp_name = [exp_name[0], exp_name[index_env-1], exp_name[index_env], "cc_False"]
    if exp_name[1] == "sarl":
        exp_name[1]= "owf"
    return exp_name

def dict_results_from_json(test_json_files, method, env1, env2, cc):
    dict_all=[]

    for i in test_json_files:
        exp_name=exp_name_from_file(i)

        if cc and "cc_True" in i:
            pass
        elif not cc and ("cc_False" in i or "cc" not in i):
            pass
        else:
            continue

        if exp_name[0] == method and exp_name[1] == env1 and exp_name[2] == env2:
            exp_name = "_".join(exp_name)

            # Remove dqn too much
            if exp_name.startswith("dqn"):
                # ignore uc/ic 10 50 100 and owf 5 25 50
                if "10" in k or "50" in k or "100" in k or ("owf" in k and ('5' in k or '25' in k or '50' in k)):
                    continue
        else:
            continue
        try:
            with open(i, "r") as file:
                infos = json.load(file)
                dict_exp= {}
                dict_exp["name"] = os.path.basename(os.path.dirname(i))
                dict_exp["test_return_mean"] = [i["value"] for i in infos["test_return_mean"]]
                dict_exp["test_return_mean_T"] = infos["test_return_mean_T"]
                dict_all.append(dict_exp)
        except Exception as e:
            print("Exception",i, e)
    return dict_all

methods = ["qmix", "qvmix","qplex","coma", "facmac", "iql",  "dqn"]
env1 = ["uc", "c", "owf"]
env2 = {"uc":["3", "5", "10", "50", "100" ], "c":["3", "5", "10", "50", "100"], "owf":["1","2","5","25","50"]}
cc = [True, False]
results={}
for cc_ in cc:
    for i in methods:
        for j in env1:
            for k in env2[j]:
                exp_name = f"{i}_{j}_{k}_cc_{cc_}"
                results[exp_name] = dict_results_from_json(test_json_files, i,j,k, cc_)

for k, v in results.items():
    print(k, len(v))
print(results.keys())

for j in env1:
    for k in env2[j]:
        print(j+k, end=" & ")

In [ ]:
import numpy as np

methods = ["facmac"] # ["qmix", "qvmix","qplex","coma", "facmac", "iql",  "dqn"]
env1 = ["uc", "c", "owf"]
env2 = {"uc":["100" ], "c":["100"], "owf":["50"]} # {"uc":["3", "5", "10", "50", "100" ], "c":["3", "5", "10", "50", "100"], "owf":["1","2","5","25","50"]}
cc = [True, False]
# Rewrite the loop but only keep one max instead of one print for each exp.
for i in methods:
    for j in env1:
        for cc_str in ["cc_False", "cc_True"]:
            for k in env2[j]:
                exp_name = f"{i}_{j}_{k}_{cc_str}"
                cur_max=-100000
                cur_max_T = None
                cur_max_name = None
                for exp in results[exp_name]:
                    argmax = np.argmax(exp["test_return_mean"][:81])
                    if exp["test_return_mean"][argmax] > cur_max:
                        cur_max = exp["test_return_mean"][argmax]
                        cur_max_T = exp["test_return_mean_T"][argmax]
                        cur_max_name = exp["name"]
                print(f"{cur_max:.2f} & {cur_max_T} & {cur_max_name}")